In [1]:
import random, math

# City coordinates
cities = {0: (0, 0), 1: (1, 5), 2: (5, 2), 3: (6, 6), 4: (8, 3)}

# Distance and pheromone initialization
distance = {(i, j): math.dist(cities[i], cities[j]) for i in cities for j in cities if i != j}
pheromone = {edge: 1.0 for edge in distance}

# Parameters
num_ants, num_iters = 10, 100
alpha, beta = 1.0, 5.0
evaporation, Q = 0.5, 100
best_path, best_length = None, float("inf")

# Choose next city based on probability
def choose_next(curr, visited):
    probs = [(c, (pheromone[(curr, c)] ** alpha) * ((1 / distance[(curr, c)]) ** beta))
             for c in cities if c not in visited]
    total = sum(p for _, p in probs)
    r = random.uniform(0, total)
    s = 0
    for city, prob in probs:
        s += prob
        if r <= s: return city
    return probs[-1][0]  # fallback

# Build a complete tour
def build_tour():
    path = [random.choice(list(cities))]
    visited = set(path)
    while len(path) < len(cities):
        next_city = choose_next(path[-1], visited)
        path.append(next_city)
        visited.add(next_city)
    return path + [path[0]]  # return to start

# Main ACO loop
for _ in range(num_iters):
    all_tours = []
    for _ in range(num_ants):
        t = build_tour()
        l = sum(distance[(t[i], t[i+1])] for i in range(len(t)-1))
        all_tours.append((t, l))
        if l < best_length: best_path, best_length = t, l
    for edge in pheromone: pheromone[edge] *= (1 - evaporation)
    for path, length in all_tours:
        for i in range(len(path) - 1):
            a, b = path[i], path[i+1]
            pheromone[(a, b)] += Q / length
            pheromone[(b, a)] += Q / length  # undirected

# Output result
print("\nBest Tour Found:")
print(" -> ".join(map(str, best_path)))
print(f"Total Distance: {best_length:.2f}")



Best Tour Found:
1 -> 0 -> 2 -> 4 -> 3 -> 1
Total Distance: 22.35


In [ ]:
'''Sure! Here's a **line-by-line explanation** of the Ant Colony Optimization (ACO) code you provided to solve the **Traveling Salesman Problem (TSP)** using Python:

---

### ✅ **CODE EXPLANATION**

---

```python
import random, math
```

* **Imports** the `random` module for randomness and `math` for mathematical functions like Euclidean distance.

---

```python
# City coordinates
cities = {0: (0, 0), 1: (1, 5), 2: (5, 2), 3: (6, 6), 4: (8, 3)}
```

* **Defines a dictionary** with 5 cities as keys and their `(x, y)` coordinates as values.

---

```python
# Distance and pheromone initialization
distance = {(i, j): math.dist(cities[i], cities[j]) for i in cities for j in cities if i != j}
```

* **Calculates and stores** the distance between each pair of cities using Euclidean distance.
* The result is stored in a dictionary with keys as `(i, j)` city pairs and values as distances.

---

```python
pheromone = {edge: 1.0 for edge in distance}
```

* **Initializes pheromone level** for each path (edge) between cities to 1.0.

---

```python
# Parameters
num_ants, num_iters = 10, 100
alpha, beta = 1.0, 5.0
evaporation, Q = 0.5, 100
best_path, best_length = None, float("inf")
```

* **`num_ants`**: Number of ants to simulate per iteration.
* **`num_iters`**: Number of iterations (generations).
* **`alpha`**: Importance of pheromone in decision making.
* **`beta`**: Importance of heuristic (distance) information.
* **`evaporation`**: Controls how quickly pheromones evaporate.
* **`Q`**: Constant used in pheromone updating.
* **`best_path` / `best_length`**: Tracks the shortest tour found so far.

---

```python
# Choose next city based on probability
def choose_next(curr, visited):
```

* **Starts definition** of a function to choose the next city.

---

```python
    probs = [(c, (pheromone[(curr, c)] ** alpha) * ((1 / distance[(curr, c)]) ** beta))
             for c in cities if c not in visited]
```

* For all unvisited cities, calculate the **probability score** based on:

  * **pheromone** strength: `(pheromone^(alpha))`
  * **visibility**: `(1/distance)^(beta)`

---

```python
    total = sum(p for _, p in probs)
```

* **Sums** all the probability scores to normalize them.

---

```python
    r = random.uniform(0, total)
```

* **Generates a random number** between 0 and the total probability — used for **roulette wheel selection**.

---

```python
    s = 0
    for city, prob in probs:
        s += prob
        if r <= s: return city
```

* Adds probabilities sequentially.
* Returns the city where the cumulative sum **passes the random threshold** `r`.

---

```python
    return probs[-1][0]  # fallback
```

* **Fallback option**: in case of rounding errors or floating-point issues, return the last city.

---

```python
# Build a complete tour
def build_tour():
```

* **Defines a function** to create a full tour starting from a random city.

---

```python
    path = [random.choice(list(cities))]
    visited = set(path)
```

* **Randomly picks a starting city**.
* Marks that city as visited.

---

```python
    while len(path) < len(cities):
        next_city = choose_next(path[-1], visited)
        path.append(next_city)
        visited.add(next_city)
```

* While not all cities are visited:

  * Choose the next city using `choose_next`
  * Add it to the path and mark it as visited.

---

```python
    return path + [path[0]]  # return to start
```

* **Returns to the starting city** to complete the tour (cycle).

---

```python
# Main ACO loop
for _ in range(num_iters):
```

* Repeats the entire ant simulation process for `num_iters` iterations.

---

```python
    all_tours = []
    for _ in range(num_ants):
```

* For each iteration, simulate the movement of `num_ants` ants.

---

```python
        t = build_tour()
        l = sum(distance[(t[i], t[i+1])] for i in range(len(t)-1))
```

* **Builds a tour** and **calculates its length**.

---

```python
        all_tours.append((t, l))
        if l < best_length: best_path, best_length = t, l
```

* **Stores the tour** and its length.
* Updates the best path if the current one is shorter.

---

```python
    for edge in pheromone: pheromone[edge] *= (1 - evaporation)
```

* **Evaporates pheromones** to reduce their intensity over time.

---

```python
    for path, length in all_tours:
        for i in range(len(path) - 1):
            a, b = path[i], path[i+1]
            pheromone[(a, b)] += Q / length
            pheromone[(b, a)] += Q / length  # undirected
```

* For each tour:

  * For every edge in the tour:

    * **Add pheromone proportional to tour quality**: `Q / length`
    * Since the graph is undirected, update both `(a, b)` and `(b, a)`.

---

```python
# Output result
print("\nBest Tour Found:")
print(" -> ".join(map(str, best_path)))
print(f"Total Distance: {best_length:.2f}")
```

* **Prints the best tour** and its total distance.

---

Let me know if you'd like the **theory part** and **expected viva questions** next.

🐜 ANT COLONY OPTIMIZATION (ACO) – THEORY
Definition:
Ant Colony Optimization is a probabilistic technique used to solve combinatorial optimization problems, inspired by the behavior of real ants finding the shortest path between their colony and food sources using pheromone trails.

✳️ BASIC CONCEPTS:
Artificial Ants:

Simulated agents that move through the problem space.

They build solutions step by step (e.g., visiting cities in TSP).

Pheromone Trails:

Each path has a pheromone value indicating its desirability.

More ants using a path → more pheromone → more likely other ants will choose it.

Heuristic Information:

Based on problem-specific knowledge (e.g., inverse of distance in TSP).

Probabilistic Transition:

The next move of an ant depends on:

Pheromone strength (α)

Heuristic value (β)

Formula:

𝑃
𝑖
𝑗
=
(
𝜏
𝑖
𝑗
)
𝛼
⋅
(
𝜂
𝑖
𝑗
)
𝛽
∑
(
𝜏
𝑖
𝑘
)
𝛼
⋅
(
𝜂
𝑖
𝑘
)
𝛽
P 
ij
​
 = 
∑(τ 
ik
​
 ) 
α
 ⋅(η 
ik
​
 ) 
β
 
(τ 
ij
​
 ) 
α
 ⋅(η 
ij
​
 ) 
β
 
​
 
where:

𝜏
𝑖
𝑗
τ 
ij
​
 : pheromone on edge 
𝑖
𝑗
ij

𝜂
𝑖
𝑗
η 
ij
​
 : 1/distance or cost

α, β: control importance

Pheromone Update:

After all ants complete tours:

Evaporation: Reduces pheromone on all paths.

Deposit: Adds pheromone to paths used by best ants.

✅ STEPS IN ACO:
Initialize pheromones for all edges.

For several iterations:

Each ant builds a solution (e.g., a tour in TSP).

Evaluate the solution quality (length, cost, etc.).

Update pheromones.

Return the best solution found.

🧠 ADVANTAGES:
Works well on NP-Hard problems like TSP, Scheduling, Routing.

Good at exploration and convergence.

Can be easily combined with other algorithms.

⚙️ COMMON PARAMETERS:
Parameter	Meaning
α (alpha)	Influence of pheromone
β (beta)	Influence of distance (heuristic info)
ρ (rho)	Evaporation rate
Q	Pheromone deposit constant
num_ants	Number of ants per iteration
num_iterations	Total simulation rounds
'''



Best Tour Found:
0 -> 2 -> 4 -> 3 -> 1 -> 0
Total Distance: 22.35
